In [1]:
import requests
import os

# Azure API 호출을 위한 설정
subscription_key = "fc46077d8bd240d9a1a0597020a2f8d4"
endpoint = "https://eastus.api.cognitive.microsoft.com/speechtotext/v3.1/transcriptions"  # 엔드포인트 주소

headers = {
    'Ocp-Apim-Subscription-Key': subscription_key
}

# 각 작업의 결과 가져오기
response = requests.get(endpoint, headers=headers)
transcriptions = response.json()

# 응답 데이터 출력
print(transcriptions)

{'values': [{'self': 'https://eastus.api.cognitive.microsoft.com/speechtotext/v3.1/transcriptions/5b12beaf-88ea-4e16-835a-e9f2353bca5e', 'model': {'self': 'https://eastus.api.cognitive.microsoft.com/speechtotext/v3.1/models/base/5b9a1565-cbfc-4e89-8033-785f00d20cde'}, 'links': {'files': 'https://eastus.api.cognitive.microsoft.com/speechtotext/v3.1/transcriptions/5b12beaf-88ea-4e16-835a-e9f2353bca5e/files'}, 'properties': {'diarizationEnabled': False, 'wordLevelTimestampsEnabled': False, 'displayFormWordLevelTimestampsEnabled': True, 'channels': [0, 1], 'punctuationMode': 'Dictated', 'profanityFilterMode': 'Masked', 'duration': 'PT1M5S'}, 'lastActionDateTime': '2024-08-21T10:18:40Z', 'status': 'Succeeded', 'createdDateTime': '2024-08-21T10:18:19Z', 'locale': 'en-US', 'displayName': '20240821_191815', 'description': 'Speech Studio Batch speech to text', 'customProperties': {'PortalAPIVersion': '3'}}, {'self': 'https://eastus.api.cognitive.microsoft.com/speechtotext/v3.1/transcriptions/45

In [2]:
## 디렉토리만 변경해서 설정하기

import requests
import os
import re
import csv

# Azure API 호출을 위한 설정
subscription_key = "fc46077d8bd240d9a1a0597020a2f8d4"
endpoint = "https://eastus.api.cognitive.microsoft.com/speechtotext/v3.1/transcriptions"  # 엔드포인트 주소

headers = {
    'Ocp-Apim-Subscription-Key': subscription_key
}

# 결과 저장할 디렉토리 설정
output_dir = "./csv/사고및보상문의_추가데이터_csv"
os.makedirs(output_dir, exist_ok=True)

# 각 작업의 결과 가져오기
response = requests.get(endpoint, headers=headers)
transcriptions = response.json()

batch_size = 50
all_transcriptions = transcriptions['values']



for i in range(0, len(all_transcriptions), batch_size):
    batch_transcriptions = all_transcriptions[i:i + batch_size]
    
    for transcription in batch_transcriptions:
        files_url = transcription['links']['files']

        # 파일 URL에서 텍스트 파일을 가져오기
        files_response = requests.get(files_url, headers=headers)
        files = files_response.json()

        for file in files['values']:
            if file['kind'] == 'Transcription':
                result_url = file['links']['contentUrl']
                result_response = requests.get(result_url)
                result_json = result_response.json()

                # 텍스트만 추출
                if 'combinedRecognizedPhrases' in result_json:
                    result_text = result_json['combinedRecognizedPhrases'][0]['display']
                    # 파일 이름으로 사용할 제목 추출 (첫 번째 문장을 제목으로 사용)
                    title = result_text.split('.')[0].strip()
                    # 파일 이름을 위해 불필요한 문자 제거
                    valid_filename = re.sub(r'[<>:"/\\|?*]', '', title)
                else:
                    result_text = ""
                    valid_filename = "no_title"

                # 텍스트를 .이나 ?로 분할
                sentences = re.split(r'([.?\n])', result_text)  # 문장을 .이나 ?로 분할하면서 구분자도 유지
                speaker_flag = True  # True이면 고객, False이면 상담사
               
                processed_lines = []
                
                for i in range(0, len(sentences), 2):
                    sentence = sentences[i].strip() + (sentences[i + 1] if i + 1 < len(sentences) else "")
                    
                    # 화자 구분 및 큰따옴표 제거
                    if speaker_flag:
                        processed_lines.append(f"고객: {sentence.replace('\"', '')}")
                    else:
                        processed_lines.append(f"상담사: {sentence.replace('\"', '')}")
                    speaker_flag = not speaker_flag  # 화자 번갈아가며 바꾸기

                # CSV 파일로 저장
                output_file = os.path.join(output_dir, f"{valid_filename}_batch.csv")
                with open(output_file, 'w', encoding='utf-8', newline='') as f:
                    writer = csv.writer(f)
                    for line in processed_lines:
                        writer.writerow([line])  # 한 줄씩 저장

                print(f"Saved transcription to {output_file}")



Saved transcription to ./csv/사고및보상문의_추가데이터_csv\I I_batch.csv
Saved transcription to ./csv/사고및보상문의_추가데이터_csv\I I_batch.csv
Saved transcription to ./csv/사고및보상문의_추가데이터_csv\공공은행입니다_batch.csv
Saved transcription to ./csv/사고및보상문의_추가데이터_csv\공 공 은행입니다_batch.csv
Saved transcription to ./csv/사고및보상문의_추가데이터_csv\공공은행입니다_batch.csv
Saved transcription to ./csv/사고및보상문의_추가데이터_csv\국민연금 납입내역을 확인할 수 있죠_batch.csv
Saved transcription to ./csv/사고및보상문의_추가데이터_csv\결제 방법을 통장인출로 바꿨는데 통장에서도 인출되고 카드도 결제됐는데 어떻게 해야 할까요 네, 이중 결제가 됐네요_batch.csv
Saved transcription to ./csv/사고및보상문의_추가데이터_csv\법인 통장 거래 내역 확인 좀 하려고 하는데요_batch.csv
Saved transcription to ./csv/사고및보상문의_추가데이터_csv\제가 담당이 새로 바뀌어서요_batch.csv
Saved transcription to ./csv/사고및보상문의_추가데이터_csv\신용불량자는 통장 개설 되나요 잔고 압류 안 당하고요_batch.csv
Saved transcription to ./csv/사고및보상문의_추가데이터_csv\잔고 증명서 발급 방법은 어떻게 되나요 지점을 방문하시거나 온라인으로 발급 신청하실 수 있습니다_batch.csv
Saved transcription to ./csv/사고및보상문의_추가데이터_csv\ATM 에서 가능한 거래는 어떤 것들이 있나요 다양한 업무가 가능하십니다_batch.csv
Saved transcription to ./csv/사고